# 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# 20 输入特征，256线性层之后的输出特征；输出特征也是下一层的输入特征数；
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1206,  0.0104,  0.1751, -0.1819, -0.0941,  0.1724, -0.1737,  0.2219,
          0.1153, -0.0852],
        [ 0.0916,  0.1644,  0.1013, -0.1384, -0.0599, -0.0059, -0.1495,  0.2479,
          0.1370, -0.0191]], grad_fn=<AddmmBackward0>)

## 自定义块

In [4]:
# 自定义一个MLP；
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.4841,  0.0017,  0.3030,  0.0601,  0.3415, -0.0062, -0.1639,  0.1265,
         -0.1160,  0.3818],
        [-0.4571,  0.0191,  0.1765,  0.0348,  0.2829,  0.0665, -0.1169,  0.2575,
          0.0092,  0.3055]], grad_fn=<AddmmBackward0>)

## 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
            print(str(idx))

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

0
1
2


tensor([[ 0.0341, -0.1112,  0.0612,  0.1057, -0.3129,  0.1261, -0.0573,  0.0412,
         -0.0237,  0.0511],
        [ 0.0534, -0.0369,  0.1187,  0.0232, -0.3427,  0.1436, -0.1232, -0.0043,
         -0.0775,  0.1310]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

继承了nn.Module类，可以很灵活的实习各种想法；

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2018, grad_fn=<SumBackward0>)

我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0500, grad_fn=<SumBackward0>)